In [1]:
# 导包
from utils.query import query
from snownlp import SnowNLP
import csv
import os
import csv
import joblib
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [2]:
# 从数据库读取数据
def getAllCommentData():
    commentList=query('select * from all_reviews',[],'select')
    return commentList

In [3]:
# 根据snow库进行标注
def targetFile():
    targetFile='target.csv'
    commentList=getAllCommentData()

    rateData=[]
    good=0
    bad=0
    middle=0

    for index,i in enumerate(commentList):
        value=SnowNLP(i[6]).sentiments
        if value>0.5:
            good+=1
            rateData.append([i[6],'积极'])
        elif value==0.5:
            middle+=1
            rateData.append([i[6],'中性'])
        elif value<0.5:
            bad+=1
            rateData.append([i[6],'消极'])

    with open(targetFile,'a+',encoding='utf8',newline='') as f:
        writer=csv.writer(f)
        writer.writerows(rateData)

In [4]:
targetFile()

In [5]:
# 从上面的文件取得数据
def getSentiment_data():
    sentiment_data=[]
    with open('target.csv','r',encoding='utf8') as readerFile:
        reader=csv.reader(readerFile)
        for data in reader:
            sentiment_data.append(data)
    return sentiment_data

In [6]:
# 模型训练
def model_train():
    sentiment_data=getSentiment_data()
    df=pd.DataFrame(sentiment_data,columns=['text','sentiment'])

    # 清洗
    df['text'] = df['text'].str.replace('\n', '')
    df['text'] = df['text'].str.replace('\t', '')
    df['text'] = df['text'].apply(lambda x: re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9\s\.,!?，。！？]', '', str(x)))

    train_data,test_data=train_test_split(df,test_size=0.2,random_state=42)

    vectorize=TfidfVectorizer()

    X_train=vectorize.fit_transform(train_data['text'])
    y_train=train_data['sentiment']
    X_test=vectorize.transform(test_data['text'])
    y_test=test_data['sentiment']

    classifier=MultinomialNB()
    classifier.fit(X_train,y_train)

    y_pred=classifier.predict(X_test)

    accuracy=accuracy_score(y_test,y_pred)

    joblib.dump(classifier, 'sentiment_model.joblib')

    print(accuracy)

In [7]:
model_train()

0.6987242394504416
